In [ ]:
# Load data into Pandas dataframe
df = pd.read_csv("526892.csv", encoding="cp1252", index_col=0)
print("DataFrame:", df)

In [ ]:
'''
from datasets import Dataset
from datasets import load_dataset
import random


def read_csv_file(file_path):
    return pd.read_csv(file_path, header=0, index_col=0, encoding="utf-8", delimiter="\t", quoting=csv.QUOTE_NONE, on_bad_lines="skip", engine='python')

train_df = read_csv_file("COVID_Train_Set.csv")
print(train_df)
test_df = read_csv_file("COVID_Test_Set.csv")
print(test_df)

data_files = {"train": train_df, "test": test_df}
print(data_files)

covid_dataset = Dataset.from_dict(data_files)
print(covid_dataset)

# Convert the train data list to a dataset object
covid_dataset["train"] = Dataset.from_dict(covid_dataset["train"])

# Shuffle and select the first 1000 samples
#covid_sample = covid_dataset["train"].shuffle(seed=42).select(range(1000))

# Convert the dataset object back to a list if necessary
#covid_sample = covid_dataset.to_pydict()["sentence"]
'''

In [ ]:

'''from datasets import load_dataset
from datasets import Dataset
import random

data_files = {"train": "COVID_Train_Set.csv", "test": "COVID_Test_Set.csv"}
covid_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

# Convert the train data list to a dataset object
covid_dataset["train"] = Dataset.from_dict(covid_dataset["train"])

# Shuffle and select the first 1000 samples
covid_sample = covid_dataset["train"].shuffle(seed=42).select(range(1000))

# Convert the dataset object back to a list if necessary
covid_sample = covid_sample.to_pydict()["text"]

# Peek at the first few examples
'''

In [ ]:
'''from datasets import Dataset
from datasets import load_dataset
import random

def read_csv_file(file_path):
    return pd.read_csv(file_path, header=0, index_col=0, encoding="utf-8", delimiter="\t", quoting=csv.QUOTE_NONE, on_bad_lines="skip", engine='python')

train_df = read_csv_file("COVID_Train_Set.csv")
test_df = read_csv_file("COVID_Test_Set.csv")

data_files = {"train": train_df, "test": test_df}

covid_dataset = Dataset.from_dict(data_files)

# Convert the train data list to a dataset object
covid_dataset[0]["train"] = Dataset.from_dict(covid_dataset[0]["train"])
'''

In [ ]:
# Process sentences as sequences
sentences = df.iloc[0:2, 0]
sequences = sentences.tolist()
print(sequences)

NameError: ignored

In [ ]:
# Define tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(tokenizer)

# Define tokenize function
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


NameError: ignored

In [ ]:
# Use a CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

# Setup the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint).to(device)
#model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.classifier = nn.Linear(model.classifier.in_features, 1).to(device)

# Freeze all parameters except the final layer
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

# Print the modified model architecture
print(model)

In [ ]:
# Convert label list to tensor and one-hot encode it
labels = torch.tensor(list(df.iloc[:5, 1]))
print("Labels:", labels)
labels_one_hot = torch.nn.functional.one_hot(labels.long(), num_classes=2)
print("Labels One Hot", labels_one_hot)



NameError: ignored

In [ ]:
# Convert data to Dataset format 1
train_data_dict = {"sentence": list(df.iloc[:5, 0]), "label": list(df.iloc[:5, 1])}
print("Train Data Dict:", train_data_dict)
dev_data_dict = {"sentence": list(df.iloc[5:7, 0]), "label": list(df.iloc[5:7, 1])}
print("Dev Data Dict:", train_data_dict)
test_data_dict = {"sentence": list(df.iloc[7:10, 0]), "label": list(df.iloc[7:10, 1])}
print("Test Data Dict:", train_data_dict)
train_dataset = Dataset.from_dict(train_data_dict)
print("Train Dataset:", train_dataset)
dev_dataset = Dataset.from_dict(dev_data_dict)
print(dev_dataset)
test_dataset = Dataset.from_dict(test_data_dict)
print(test_dataset)

# Convert data to Dataset format 2
data_dict = {"sentence": list(df.iloc[0:2, 0]), "label": list(df.iloc[0:2, 1])}
raw_dataset = Dataset.from_dict(data_dict)

# Create dictionary with "sentence1" and "label" as keys and the respective columns from the pandas dataframe as values
data_dict = {"sentence": list(df.iloc[0:10, 0]), "label": list(df.iloc[0:10, 1])}

# Convert dictionary to Dataset object 2
dataset = Dataset.from_dict(data_dict)
print (dataset)

In [ ]:
# Tokenize sentences
batch = tokenizer(list(df.iloc[0:10, 0]), padding=True, truncation=True, return_tensors="pt").to(device)
print(batch)

# Tokenize sentences
train_tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
print(train_tokenized_dataset)
dev_tokenized_dataset = dev_dataset.map(tokenize_function, batched=True)
print(dev_tokenized_dataset)
test_tokenized_dataset = test_dataset.map(tokenize_function, batched=True)
print(test_tokenized_dataset)


In [ ]:
# Add labels to batch
batch["labels"] = torch.tensor(df.iloc[0:10, 1].values).unsqueeze(1).to(device)  # Add unsqueeze here
#batch["labels"] = torch.tensor(df.iloc[0:10, 1].values).to(device)  # Add unsqueeze here

# This is a new one
batch["labels"] = torch.tensor([1] * len(sequences))
print(batch["labels"])

NameError: ignored

In [ ]:
# Set output format
columns_to_return = ["input_ids", "attention_mask", "label"]
train_tokenized_dataset.set_format(type='torch', columns=columns_to_return)
print(train_tokenized_dataset)
dev_tokenized_dataset.set_format(type='torch', columns=columns_to_return)
print(dev_tokenized_dataset)
test_tokenized_dataset.set_format(type='torch', columns=columns_to_return)
print(test_tokenized_dataset)

NameError: ignored

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
print(data_collator)

NameError: ignored

In [ ]:
batch_size = 8
train_dataloader = torch.utils.data.DataLoader(train_tokenized_dataset, batch_size=batch_size, collate_fn=data_collator)
dev_dataloader = torch.utils.data.DataLoader(dev_tokenized_dataset, batch_size=batch_size, collate_fn=data_collator)
test_dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=batch_size, collate_fn=data_collator)

NameError: ignored

In [1]:
batch_size = 1
dataloader = torch.utils.data.DataLoader(tokenized_dataset, batch_size=batch_size)
print(dataloader)
for batch in dataloader:
    input_ids = batch["input_ids"]
    print(input_ids)
    attention_mask = batch["attention_mask"]
    print(attention_mask)
    label = batch["label"]
    print(label)
    inputs = {"input_ids": input_ids, "attention_mask": attention_mask, "labels": label.to(torch.long), "targets": targets}
    print(inputs)
    outputs = model(**inputs)
    print(outputs)
    loss = outputs.loss
    print(loss)
    logits = outputs.logits
    print(logits)

NameError: ignored

In [ ]:
optimizer = AdamW(model.parameters())
#output = model(**batch).logits.squeeze(1)  # Squeeze the output to get [batch_size]
output = model(**batch).logits  # Squeeze the output to get [batch_size]
loss = nn.BCEWithLogitsLoss()(output, batch["labels"].float())
loss.backward()
optimizer.step()

In [ ]:
# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    metric = load_metric("f1")
    return metric.compute(predictions=preds, references=labels)

In [ ]:
#training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "epoch",   # evaluation strategy to adopt during training
    learning_rate=2e-5,             # learning rate
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    num_train_epochs=3,             # total number of training epochs
    weight_decay=0.01,              # strength of weight decay
    push_to_hub=False,
)
print(training_args)

In [ ]:
trainer = Trainer(
    model=model,                     
    args=training_args,              
    train_dataset=train_tokenized_dataset,         
    eval_dataset=dev_tokenized_dataset,            
    data_collator=data_collator,                     
    tokenizer=tokenizer,                            
    compute_metrics=compute_metrics,                  
)
print(trainer)

In [ ]:
# Evaluate the model on the test set
#predictions = trainer.predict(test_dataset)
predictions = trainer.predict(tokenized_datasets["test"])
#print(predictions.predictions.shape, predictions.label_ids.shape)
preds = np.argmax(predictions.predictions, axis=-1)
test_results = trainer.evaluate(test_tokenized_dataset)
print(test_results)

In [ ]:
# Compute and print the F1 score
metric = load_metric("f1")
predictions = trainer.predict(test_tokenized_dataset)
preds = predictions.predictions.argmax(-1)
f1_score = metric.compute(predictions=preds, references=predictions.label_ids)
print(f1_score)